In [14]:
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.combine import SMOTEENN
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [10]:
cleaned_data_url = '../../Resources/2018MBB_StatsAndDraft_Cleaned.csv'

raw_df = pd.read_csv(cleaned_data_url)

numerical_cols = raw_df.dtypes[raw_df.dtypes != 'object'].index.tolist()

numerical_df = raw_df[numerical_cols]
numerical_df.index = raw_df['Player']

X = numerical_df.drop(columns=['Pk'])
y = numerical_df['Pk'].apply(lambda x: 1 if x <= 60 else 2)

In [11]:
smoteenn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smoteenn.fit_resample(X,y)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled,random_state=1)

classifier = LogisticRegression(solver='lbfgs',random_state=1, max_iter=1000)
classifier.fit(X_train,y_train)

lr_pred = classifier.predict(X_test)

lr_acc_score = accuracy_score(y_test,lr_pred)
lr_matrix = confusion_matrix(y_test,lr_pred)
lr_results = pd.DataFrame({"Prediction": lr_pred, "Actual": y_test}).reset_index(drop=True)
lr_report = classification_report(y_test,lr_pred,zero_division=True)

In [19]:
print(lr_matrix)
print(lr_report)

[[458  33]
 [ 53 349]]
              precision    recall  f1-score   support

           1       0.90      0.93      0.91       491
           2       0.91      0.87      0.89       402

    accuracy                           0.90       893
   macro avg       0.90      0.90      0.90       893
weighted avg       0.90      0.90      0.90       893



In [15]:
rf = RandomForestClassifier(n_estimators=120,random_state=1)
rf.fit(X_train,y_train)

rf_pred = rf.predict(X_test)

rf_acc_score = accuracy_score(y_test,rf_pred)
rf_matrix = confusion_matrix(y_test,rf_pred)
rf_results = pd.DataFrame({"Prediction": rf_pred, "Actual": y_test}).reset_index(drop=True)
rf_report = classification_report(y_test,rf_pred,zero_division=True)

In [20]:
print(rf_matrix)
print(rf_report)

[[488   3]
 [  9 393]]
              precision    recall  f1-score   support

           1       0.98      0.99      0.99       491
           2       0.99      0.98      0.98       402

    accuracy                           0.99       893
   macro avg       0.99      0.99      0.99       893
weighted avg       0.99      0.99      0.99       893



In [21]:
importances = rf.feature_importances_
features = X.columns
feature_importances = sorted(zip(importances,features),reverse=True)

In [22]:
feature_importances

[(0.13749463168119358, 'FGM'),
 (0.09928689413143799, 'PPG'),
 (0.08891432818296253, 'MPG'),
 (0.08070328207504095, 'DRB'),
 (0.07823615531428209, 'FGA'),
 (0.05658736532147858, 'GP'),
 (0.05012630644733565, 'FG%'),
 (0.04522882567846539, 'FTA'),
 (0.043954980807591214, 'BPG'),
 (0.043482714195805905, 'FTM'),
 (0.04118152452677147, 'PF'),
 (0.03824726589884613, 'APG'),
 (0.035845437519958195, 'RPG'),
 (0.030566178491589786, 'ORB'),
 (0.02578444987542877, '3PA'),
 (0.02431641902209148, '3PM'),
 (0.023827140474352363, 'FT%'),
 (0.022168483028746844, '3P%'),
 (0.018540181867432944, 'SPG'),
 (0.015507435459188274, 'TOV')]

In [34]:
cutoff = 0.7
total = 0
features_used = 0

for feature in feature_importances:
    if total <= cutoff:
        total += feature[0]
        features_used += 1

features_used

10